In [1]:
config_file = 'delineation/configs/dislocation_matching_disp_and_warp_and var_joint_home.yml'
aug_config_file = 'delineation/configs/aug.yml'
import torch
from delineation.configs.defaults_segmentation import _C as cfg
from delineation.datasets import make_data_loader
from delineation.models import build_model_list
from delineation.utils import settings, cost_volume_helpers
from delineation.utils.settings import evaluate_results
import cv2
from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt
import torch.nn.functional as F

import yaml

cfg.merge_from_file(config_file)
with open(aug_config_file, 'r') as ymlfile:
    cfg_aug = yaml.load(aug_config_file)

_device = settings.initialize_cuda_and_logging(cfg)  # '_device' is GLOBAL VAR

train_loader, val_loader = make_data_loader(cfg, cfg_aug)
seg_model, model = build_model_list(cfg, True)

/home/dagnyt/anaconda3/envs/fast2/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)
/home/dagnyt/EPFL/dislocations/delineation/layers/loss.py:5: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  matplotlib.use('agg')
/home/dagnyt/EPFL/dislocations/delineation/utils/settings.py:8: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  matplotlib.use('agg')


Using cuda
unet-efficient
unet-efficient-effnet
EfficientNet-b0
=> loaded checkpoint '/home/dagnyt/EPFL/dislocations/models/models_2020_05/matching/dislocations_matching_disp_warp_var_joint_32/150_seg.tar' (epoch 150)
scmnet
=> loaded checkpoint '/home/dagnyt/EPFL/dislocations/models/models_2020_05/matching/dislocations_matching_disp_warp_var_joint_32/150_scmnet_light.tar' (epoch 150)


In [3]:
seg_model.eval()
model.eval()

l = cv2.imread('/home/dagnyt/EPFL/dislocations/eval_data/2019-12-02_TiAl_box3n15_SIFT_PNG/Aligned_9_of_90004.png',0)
r = cv2.imread('/home/dagnyt/EPFL/dislocations/eval_data/2019-12-02_TiAl_box3n15_SIFT_PNG/Aligned_9_of_90005.png',0)

l_name = '2019-12-02_TiAl_box3n15_SIFT_PNG'
l = cv2.resize(l, (512,512))
r = cv2.resize(r, (512,512))

l = torch.from_numpy(l).unsqueeze(0).unsqueeze(0)/255.0
r = torch.from_numpy(r).unsqueeze(0).unsqueeze(0)/255.0

with torch.no_grad():
            device = torch.device('cpu')
            seg_model = seg_model.to(device)

    
            indices = cost_volume_helpers.volume_indices(2 * cfg.TRAINING.MAXDISP, len(l),
                                                     cfg.TRAINING.HEIGHT, cfg.TRAINING.WIDTH, _device)

            l_segmap, l_seg = seg_model(l)
            r_segmap, r_seg = seg_model(r)
            
            dl_scores = model(l_segmap.cuda(), r_segmap.cuda())
            l_seg = l_seg.cpu().numpy()>0.1

            dl_ = F.softmax(-dl_scores, 2)
            dl = torch.sum(dl_.mul(indices), 2) - cfg.TRAINING.MAXDISP
            
            dl = dl.detach().permute(0, 2, 3, 1).cpu().numpy()

            for i in tqdm(range(len(dl))):
                vol = cost_volume_helpers.back_project_numpy(l_seg[i,0,:,:], dl[i,:,:,:], cfg.TRAINING.MAXDISP, mode='two-sided')

                for idx, rotation_angle in enumerate(range(-20, 21, 1)):
                    cost_volume_helpers.visualize_volume(l_name, vol[0, :, :, :],
                                                         rotation_angle,
                                                         cfg.LOGGING.LOG_DIR,
                                                         mode='scatter',
                                                         save_ext=str(idx)+'.png',
                                                         plot=False)


  0%|          | 0/1 [00:00<?, ?it/s]

[[ 1.304203    1.345478    0.3525772  ... -0.71103096 -0.15319633
  -0.39879608]
 [ 1.8426628   1.9898415   1.9350128  ... -0.52501297 -0.7460575
   0.21668243]
 [ 1.8485794   1.592659    1.7716942  ... -1.1856346  -0.7368412
  -0.6308937 ]
 ...
 [-1.2152977  -2.6910667  -2.4693794  ... -0.5077362  -0.9374504
  -0.4260254 ]
 [-0.99671555 -2.3866768  -2.2472515  ... -0.81874466 -1.1671104
  -1.7770615 ]
 [-0.1426487  -1.1984215  -2.6537933  ... -1.1863976  -0.6746235
  -0.94906235]]
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]
[[ 1.304203    1.345478    0.3525772  ... -0.71103096 -0.15319633
  -0.39879608]
 [ 1.8426628   1.9898415   1.9350128  ... -0.52501297 -0.7460575
   0.21668243]
 [ 1.8485794   1.592659    1.7716942  ... -1.1856346  -0.7368412
  -0.6308937 ]
 ...
 [-1.2152977  -2.6910667  -2.4693794  ... -0.5077362  -0.9374504
  -0.4260254 ]
 [-0.99671555 -2.3866768  -2.2472515  ... -0.81874466 -1.1671104
  

100%|██████████| 1/1 [00:37<00:00, 37.46s/it]
